In [1]:
import ipywidgets as widgets
from IPython.display import display

# Create a text input widget
text_input = widgets.Text(
    value='Hello, World!',
    placeholder='Type something',
    description='Input:',
    disabled=False
)
display(text_input)

Text(value='Hello, World!', description='Input:', placeholder='Type something')

In [2]:
import ipyaudioworklet as ipyaudio

recorder = ipyaudio.AudioRecorder()
display(recorder)

AudioRecorder()